In [21]:
from data import UCIDataModule
from diet import DIET
from architecture import MyDIETBackbone, MyDIETProjectionHead
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

In [22]:
dm = UCIDataModule(return_index_as_label=True)
output_length = len(dm.train_dataloader().dataset[:][0])

In [23]:
model = DIET(
    encoder=MyDIETBackbone(encoding_size=128),
    linear_layer=MyDIETProjectionHead(encoding_size=128, output_size=output_length),
    cosine_annealing_total_epochs=200,
    scheduler='WarmupCosineAnnealingLR'
)

In [24]:
train_data = dm.train_dataloader().dataset[:][0]
train_data.shape

torch.Size([7352, 9, 128])

In [25]:
logger = CSVLogger(save_dir='', name='pretraining-logs')
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='train_loss',
    mode='min',
    every_n_epochs=20,
    filename='{epoch:03d}',
    save_top_k=-1,
    save_last=True,
    verbose=False,
)

trainer = L.Trainer(
    max_epochs=200,
    logger=logger,
    callbacks=[checkpoint_callback],
)
trainer.fit(model=model, datamodule=dm)

Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name         | Type                 | Params | Mode 
--------------------------------------------------------------
0 | encoder      | MyDIETBackbone       | 101 K  | train
1 | linear_layer | MyDIETProjectionHead | 941 K  | train
2 | loss         | CrossEntropyLoss     | 0      | train
--------------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.169     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Epoch 199: 100%|██████████| 58/58 [00:00<00:00, 121.66it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 58/58 [00:00<00:00, 73.97it/s, v_num=0] 


In [26]:
import os
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


cp_path = 'checkpoints'
output_path = 'plots'

files = [file for file in os.listdir(cp_path) if file.startswith('epoch=') and file.endswith('.ckpt')]
files = [cp_path + '/' + path for path in sorted(files)]

dm = UCIDataModule(return_index_as_label=True)
output_length = len(dm.train_dataloader().dataset[:][0])
x, y = dm.test_dataloader().dataset[:]
print(x.shape, y.shape)
os.makedirs(output_path, exist_ok=True)


torch.Size([2947, 9, 128]) torch.Size([2947, 1])


In [28]:
for file in files:
    model = DIET.load_from_checkpoint(file, strict=True, encoder=MyDIETBackbone(encoding_size=128), linear_layer=MyDIETProjectionHead(encoding_size=128, output_size=output_length))
    print(f'Loaded model from {file}')
    x_embedded = model.encoder(x.to(model.device)).detach().cpu().numpy()
    x_2d = TSNE(n_components=2).fit_transform(x_embedded)
    epoch = file.split("/")[-1][:-5]
    # Plotting
    plt.scatter(x_2d[:, 0], x_2d[:, 1], c=y)
    plt.title(epoch)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{output_path}/{epoch}.png')
    plt.close()

Loaded model from checkpoints/epoch=019.ckpt
Epoch: epoch=019.
Loaded model from checkpoints/epoch=039.ckpt


KeyboardInterrupt: 